## Import Necessary Packages

In [ ]:
import pandas as pd
import urllib.request

## Data Cleaning

In [ ]:
week_days=set(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])
clean_data={}
index=-1

In [ ]:
# Parse log file
log_file=urllib.request.urlopen('https://raw.githubusercontent.com/kaazima/Internet-Analysis-Project/master/statistics.log')
for line in log_file:
  unclean_data=line.decode("utf-8")
  if unclean_data[:3] in week_days:
    index+=1
    clean_data[index]={}
    clean_data[index]['Day']=unclean_data[:3]
    time=unclean_data.split()[3].split(':')
    clean_data[index]['Hour']=int(time[0])
    clean_data[index]['Min']=int(time[1])
  else:
    unclean_data=unclean_data.split(':')
    prop=unclean_data[0].strip()
    remain=unclean_data[1].strip()
    if prop=='Server':
      clean_data[index][prop]=remain.split('(')[0].strip()
    elif prop=='Latency' and remain!='FAILED':
      remain=remain.split('(')
      clean_data[index][prop]=float(remain[0].strip().split()[0])
      clean_data[index]['Jitter']=float(remain[1].split()[0])
    elif (prop=='Download' or prop=='Upload') and remain!='FAILED':
      clean_data[index][prop]=float(remain.split()[0])
    elif prop=='Packet Loss' and remain!='Not available.':
      clean_data[index][prop]=float(remain.split('%')[0])

## Generate a DataFrame that contains Cleansed Data

In [ ]:
# Convert dictionary to pandas dataframe
data=pd.DataFrame.from_dict(clean_data,orient ='index')

In [ ]:
data.head(7)

,Day,Hour,Min,Server,Latency,Jitter,Download,Upload,Packet Loss
0,Tue,12,38,Airtel Broadband - Chennai,39.54,21.98,6.63,1.41,NaN
1,Tue,12,40,Airgenie - Chennai,78.52,1.77,5.50,1.36,NaN
2,Tue,12,42,Airtel Broadband - Chennai,71.26,9.71,5.39,1.70,NaN
3,Tue,12,44,TataSky Broadband Private Limited - Chennai,41.48,20.52,3.24,1.05,0.0
4,Tue,12,46,NaN,NaN,NaN,NaN,NaN,NaN
5,Tue,12,48,Cherrinet - K Net Solutions Pvt Ltd - Chennai,78.04,2.55,7.99,NaN,NaN
6,Tue,12,50,ACT Fibernet - Chennai,42.17,19.01,5.58,1.33,0.0
